## Light interception with Caribu

In progress ...  

We have to configure Caribu to simulate the same light as in STICS

In [6]:
## Imports
import math
import numpy as np
from random import *

from oawidgets.plantgl import *
from openalea.plantgl.all import Material, Color3, Shape, Scene, Viewer, Translated, AxisRotated

from openalea.archicrop.cereals import build_shoot
from openalea.archicrop.display import build_scene, display_scene
from openalea.archicrop.simulation import retrieve_stics_dynamics_from_file, generate_and_display_plant, dict_ranges_to_all_possible_combinations

# Enable plotting with PlantGL
%gui qt

# Set nice color for plants
nice_green=Color3((50,100,0))

## Pseudo-intercrop scene

In [3]:
## Imports

# Fix a seed
seed(1)

## Code for generating an intercrop from descritive parameters

def plant(height, nb_phy, max_leaf_length, wl, phyllotactic_angle):
    """ return the MTG of a cereal shoot generative from descriptive parameters """

    shoot, g = build_shoot(nb_phy=nb_phy,
                            height=height,
                            max_leaf_length=max_leaf_length,
                            wl=wl, diam_base=2.5, diam_top=1.0, 
                            insertion_angle=30, scurv=0.6, curvature=120, 
                            alpha=-2.3, stem_q=1, rmax=0.67, skew=0.05,
                            phyllotactic_angle=phyllotactic_angle,
                            phyllotactic_deviation=20)
    
    return g


# Organize the plant mixture in alternate rows
n_rows = 6 # 10
len_rows = 10

d_inter = 70
d_intra = 70

def plant_in_row(i):
     if i%(4*d_inter)==0 or i%(4*d_inter)==d_inter: return plant(height=300, nb_phy=26, max_leaf_length=80, wl=0.12, phyllotactic_angle=180)
     # if i%(2*d_inter)==0: return plant(height=1700, nb_phy=18, max_leaf_length=100, wl=0.12, phyllotactic_angle=180)
     else: return plant(height=150, nb_phy=15, max_leaf_length=40, wl=0.7, phyllotactic_angle=60)


plants_in_intercrop = [plant_in_row(x) for x in range(0, n_rows*d_inter, d_inter) for y in range(0, len_rows*d_intra, d_intra)]

positions=[(x,y,0) for x in range(0, n_rows*d_inter, d_inter) for y in range(0, len_rows*d_intra, d_intra)]

# Build and display scene
scene_ic, nump = build_scene(plants_in_intercrop, positions, leaf_material=Material(nice_green), stem_material=Material(nice_green))
# display_scene(scene_ic)
PlantGL(scene_ic)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

## Vary parameters one by one

In [4]:
# For sorghum
archi_params = dict(
    nb_phy = [10,15,20,25,30], 
    max_leaf_length = [40,50,60,70,80,90,100,110,120], 
    wl = [0.1,0.11,0.12], 
    diam_base = 2.5, 
    diam_top = 1.5, 
    insertion_angle = [10, 50], 
    scurv = [0.6, 0.8], 
    curvature = [45, 135], 
    alpha = -2.3, 
    stem_q = 1.1, 
    rmax = [0.7,0.8,0.9], 
    skew = 0.001, # 0.0005
    phyllotactic_angle = 180, 
    phyllotactic_deviation = [5, 30]
)


# algo that returns dicts where only one parameter varies

def generate_single_list_dicts(params):
    """
    Generate dictionaries where only one parameter remains a list and all others are single values.
    
    :param params: dict - The input dictionary containing parameters.
    :return: list of dict - A list of dictionaries with only one parameter as a list.
    """
    single_list_dicts = []

    for key, value in params.items():
        if isinstance(value, list):  # Only consider keys with list values
            # Create a base dictionary with single values
            base_dict = {k: (v[0] if isinstance(v, list) else v) for k, v in params.items()}
            # Replace the single value with the original list for the current key
            base_dict[key] = value
            # Add the dictionary to the results
            single_list_dicts.append(base_dict)
    
    return single_list_dicts

## Vary all parameters

In [ ]:
archi_params_combinations = dict_ranges_to_all_possible_combinations(archi_params)

par_caribu = []
par_stics = []

for archi in archi_params_combinations:
    caribu, stics = model_with_light_inter(*archi,
                                            sowing_density=sowing_density, 
                                            inter_row=inter_row,
                                            filename_outputs=filename_outputs)
    par_caribu.append(caribu)
par_stics = stics

In [ ]:
# Convert to a NumPy array for easier manipulation
curves = par_caribu
curves_array = np.array(curves)

# Calculate the envelope: min and max values for each time point
min_values = curves_array.min(axis=0)
max_values = curves_array.max(axis=0)

# Plotting the envelope along with individual curves for context
time_points = tt_cum[1:]
for curve in curves:
    plt.plot(time_points, curve, alpha=0.5, linestyle='--')  # Plot each curve (optional for visualization)

# plt.fill_between(time_points, min_values, max_values, color="skyblue", alpha=0.4)
# plt.plot(time_points, min_values, color="blue", linestyle="--", label="Min 3D")
# plt.plot(time_points, max_values, color="red", linestyle="--", label="Max 3D")
plt.plot(time_points, par_stics[2:], color="black", label="STICS")
# plt.scatter(range(len(la_cum)), la_cum)

# Labels and legend
plt.xlabel("Thermal time (°C.day)")
plt.ylabel("PAR absorbed (% of incident PAR)")
plt.title("PAR absorbed: 3D canopy vs. STICS")
plt.legend()
plt.show()

## Compute Beer-Lambert light extinction coefficient at each time step

### Light interception in STICS

The radiation intercepted by the crop $raint$ is expressed according to a Beer’s law function of $lai$. The $extinP$ parameter is a daily extinction coefficient and $parsurrgG$ is a climatic parameter corresponding to the ratio (in radiative energy) of photosynthetically active radiation to the global radiation $trg(t)$ (around 0.48, Varlet-Grancher et al. (1982)). 

$$raint(t)=0.95*parsurrgG*trg(t)*(1−exp^{−extinP*(lai(t)+eai(t))})$$

$$extinP = - \frac{1}{lai(t)+eai(t)} ln(1 - \frac{raint(t)}{0.95*parsurrgG*trg(t)})$$

$$extinP = - \frac{1}{lai(t)+eai(t)} ln(1 - ratio\_par\_abs)$$

In [ ]:
# Compute coef extinP

extinP_list = []

for par_time_series in par_caribu:
    extinP_per_sim = []
    for i,par in enumerate(par_time_series):
        if par < 1:
            lai = la_cum[i+2]*sowing_density/1000
            extinP = -1/lai * math.log(1-par)
        else:
            extinP = 0
        extinP_per_sim.append(extinP)
    extinP_list.append(extinP_per_sim)

time_points = tt_cum[1:]
for curve in extinP_list:
    plt.plot(time_points, curve, alpha=0.5, linestyle='-')
plt.show()

In [ ]:
# courbe parameters beer-lambert --> analysis of decomposition of uncertainty, parametric

## Next steps

- Configure Caribu to mimic what happens in STICS' 2.5D formalism, cf https://vezy.github.io/STICS-IC-paper/
- Simulations to run with IFB Cloud